In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# RQ1: Which technologies have been investigated in the last decade?

In [3]:
data = pd.read_csv('./data/PoPCites2.csv')

In [4]:
def CreateTermDocMatrix(data):
    documents = []

    for text in data['Abstract']:
        documents.append(text)


    count_vectorizer = CountVectorizer(stop_words='english')
    #count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(documents)

    doc_term_matrix = sparse_matrix.todense()
    df = pd.DataFrame(doc_term_matrix, columns=count_vectorizer.get_feature_names())
    df['PublishingYear'] = data['Year']
    returndf = df.groupby('PublishingYear').sum()
    return returndf

In [ ]:
#abstract['PublishingYear'] == year

abstract = CreateTermDocMatrix(data)
abstract.info()
print(abstract)

yeardata = abstract.loc[2017]

for year in range(2017,2020,1):
    yeardata = abstract.loc[year]
    #yeardata = abstract.loc['2017']
    #print(yeardata)
    yeardata.plot.bar()
    
print('Fertig!')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 2017 to 2020
Columns: 1669 entries, 12 to young
dtypes: int64(1669)
memory usage: 52.2 KB
                12  14  15  17  1951  1960s  1968  1970s  1989  1990  ...  \
PublishingYear                                                        ...   
2017             0   1   0   0     0      0     0      0     0     0  ...   
2018             0   0   0   0     0      1     1      1     1     2  ...   
2019             1   0   0   0     0      0     0      1     0     0  ...   
2020             0   0   1   1     1      0     1      0     0     0  ...   

                workers  working  world  worldwide  write  writing  year  \
PublishingYear                                                             
2017                  1        2      2          0      2        1     3   
2018                  0        0      1          0      0        0     1   
2019                  0        0      3          2      0        0     0   
2020  